<a href="https://colab.research.google.com/github/renatopuga/rnaseq-chr22/blob/main/rnaseq_chr22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNAseq Google Colab
> Prof. Renato Puga - renatopuga@gmail.com

Aula prática de `RNAseq` da Pós-graduação de Bioinformática aplicada a Genômica Médica do Hospital Albert Einstein.

# Objetivo 

Executar um pipeline de RNAseq para seis amostras simuladas*, preparando a referência do genoma, alinhando os arquivos fastqs até a tabela de contagem de reads.

Cada uma das amostras contém 30 mil reads do chr22 do genoma humano hg19.

>\* As sequências foram geradas previamente com o comando `rsem-simulate-reads` do pacote RSEM.


# STAR (Spliced Transcripts Alignment to a Reference)
> https://academic.oup.com/bioinformatics/article/29/1/15/272537

STAR é um alinhador projetado para abordar especificamente muitos dos desafios do mapeamento de dados RNA-seq usando uma estratégia para contabilizar alinhamentos emendados.



## Clonar Repositório git

Ao criar um repositório no GitHub, ele passa a existir como um repositório remoto. É possível clonar o repositório para criar uma cópia local no Google Drive e sincronizar entre os dois locais.

Removendo arquivos e diretórios anteriores.

In [1]:
%%bash
cd /content/
rm -rf rnaseq-chr22
pwd

/content


In [2]:
! git clone https://github.com/renatopuga/rnaseq-chr22

Cloning into 'rnaseq-chr22'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 158 (delta 77), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (158/158), 39.63 MiB | 23.54 MiB/s, done.
Resolving deltas: 100% (77/77), done.


## Entrando no Diretório Clonado

Após clonar o repositório git, temos que entrar no diretório e seguir alguns passos de instalação para executar o pipeline STAR.

In [3]:
%cd /content/rnaseq-chr22/

/content/rnaseq-chr22


## Download e Instalação STAR e RSEM

- mkdir -p # ignora se o diretório existe
- wget -c # volta o download a partir de onde parou.

- TAR (agrupar) GZ (compactar) # igual ao ZIP no Windows
- tar -zxvf # z: zipado; x: extrair; v: verbose f: files

## STAR download

In [4]:
%%bash
mkdir -p apps
cd apps
wget -c https://github.com/alexdobin/STAR/archive/2.7.8a.tar.gz
tar -zxvf 2.7.8a.tar.gz

STAR-2.7.8a/
STAR-2.7.8a/.gitignore
STAR-2.7.8a/.gitmodules
STAR-2.7.8a/.travis.yml
STAR-2.7.8a/CHANGES.md
STAR-2.7.8a/CODE_OF_CONDUCT.md
STAR-2.7.8a/CONTRIBUTING.md
STAR-2.7.8a/LICENSE
STAR-2.7.8a/README.md
STAR-2.7.8a/RELEASEnotes.md
STAR-2.7.8a/_config.yml
STAR-2.7.8a/bin/
STAR-2.7.8a/bin/Linux_x86_64/
STAR-2.7.8a/bin/Linux_x86_64/STAR
STAR-2.7.8a/bin/Linux_x86_64/STARlong
STAR-2.7.8a/bin/Linux_x86_64_static/
STAR-2.7.8a/bin/Linux_x86_64_static/STAR
STAR-2.7.8a/bin/Linux_x86_64_static/STARlong
STAR-2.7.8a/bin/MacOSX_x86_64/
STAR-2.7.8a/bin/MacOSX_x86_64/STAR
STAR-2.7.8a/bin/MacOSX_x86_64/STARlong
STAR-2.7.8a/doc/
STAR-2.7.8a/doc/STARmanual.pdf
STAR-2.7.8a/docs/
STAR-2.7.8a/docs/STARconsensus.md
STAR-2.7.8a/docs/STARsolo.md
STAR-2.7.8a/extras/
STAR-2.7.8a/extras/doc-latex/
STAR-2.7.8a/extras/doc-latex/STARmanual.tex
STAR-2.7.8a/extras/doc-latex/convertParDefToLatexTable.awk
STAR-2.7.8a/extras/doc-latex/parametersDefault.tex
STAR-2.7.8a/extras/docker/
STAR-2.7.8a/extras/docker/Dockerf

--2021-03-20 04:04:17--  https://github.com/alexdobin/STAR/archive/2.7.8a.tar.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/alexdobin/STAR/tar.gz/2.7.8a [following]
--2021-03-20 04:04:18--  https://codeload.github.com/alexdobin/STAR/tar.gz/2.7.8a
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘2.7.8a.tar.gz’

     0K .......... .......... .......... .......... ..........  996K
    50K .......... .......... .......... .......... .......... 1.91M
   100K .......... .......... .......... .......... .......... 1.97M
   150K .......... .......... .......... .......... .......... 51.7M
   200K .......... .......... ........

## RSEM download

In [5]:
%%bash
cd /content/rnaseq-chr22/apps/
wget -c https://github.com/deweylab/RSEM/archive/v1.3.3.tar.gz
tar -zxvf v1.3.3.tar.gz

RSEM-1.3.3/
RSEM-1.3.3/.gitignore
RSEM-1.3.3/AlignerRefSeqPolicy.h
RSEM-1.3.3/BamConverter.h
RSEM-1.3.3/BamWriter.h
RSEM-1.3.3/Buffer.h
RSEM-1.3.3/COPYING
RSEM-1.3.3/EBSeq/
RSEM-1.3.3/EBSeq/EBSeq_1.2.0.tar.gz
RSEM-1.3.3/EBSeq/KernSmooth_2.23-15.tar.gz
RSEM-1.3.3/EBSeq/Makefile
RSEM-1.3.3/EBSeq/bitops_1.0-6.tar.gz
RSEM-1.3.3/EBSeq/blockmodeling_0.1.8.tar.gz
RSEM-1.3.3/EBSeq/caTools_1.17.1.tar.gz
RSEM-1.3.3/EBSeq/calcClusteringInfo.cpp
RSEM-1.3.3/EBSeq/gdata_2.17.0.tar.gz
RSEM-1.3.3/EBSeq/gplots_2.17.0.tar.gz
RSEM-1.3.3/EBSeq/gtools_3.5.0.tar.gz
RSEM-1.3.3/EBSeq/install
RSEM-1.3.3/EBSeq/rsem-for-ebseq-find-DE
RSEM-1.3.3/EBSeq/rsem-for-ebseq-generate-ngvector-from-clustering-info
RSEM-1.3.3/EM.cpp
RSEM-1.3.3/GTFItem.h
RSEM-1.3.3/Gibbs.cpp
RSEM-1.3.3/GroupInfo.h
RSEM-1.3.3/HitContainer.h
RSEM-1.3.3/HitWrapper.h
RSEM-1.3.3/LenDist.h
RSEM-1.3.3/Makefile
RSEM-1.3.3/Model.h
RSEM-1.3.3/ModelParams.h
RSEM-1.3.3/NoiseProfile.h
RSEM-1.3.3/NoiseQProfile.h
RSEM-1.3.3/Orientation.h
RSEM-1.3.3/PairedE

--2021-03-20 04:04:20--  https://github.com/deweylab/RSEM/archive/v1.3.3.tar.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/deweylab/RSEM/tar.gz/v1.3.3 [following]
--2021-03-20 04:04:20--  https://codeload.github.com/deweylab/RSEM/tar.gz/v1.3.3
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘v1.3.3.tar.gz’

     0K .......... .......... .......... .......... ..........  979K
    50K .......... .......... .......... .......... .......... 1.90M
   100K .......... .......... .......... .......... .......... 1.88M
   150K .......... .......... .......... .......... .......... 37.4M
   200K .......... .......... .......... 

In [6]:
%%bash
cd /content/rnaseq-chr22/apps/RSEM-1.3.3
make
cd ..


g++ -std=gnu++98 -Wall -I. -I. -Isamtools-1.3/htslib-1.3  -O3 -c -o extractRef.o extractRef.cpp
g++  -o rsem-extract-reference-transcripts extractRef.o 
g++ -std=gnu++98 -Wall -I. -I. -Isamtools-1.3/htslib-1.3  -O3 -c -o synthesisRef.o synthesisRef.cpp
g++  -o rsem-synthesis-reference-transcripts synthesisRef.o 
g++ -std=gnu++98 -Wall -I. -I. -Isamtools-1.3/htslib-1.3  -O3 -c -o preRef.o preRef.cpp
g++  -o rsem-preref preRef.o 
g++ -std=gnu++98 -Wall -I. -I. -Isamtools-1.3/htslib-1.3  -O3 -c -o buildReadIndex.o buildReadIndex.cpp
g++  -o rsem-build-read-index buildReadIndex.o 
g++ -std=gnu++98 -Wall -I. -I. -Isamtools-1.3/htslib-1.3  -O3 -ffast-math -c -o simulation.o simulation.cpp
g++  -o rsem-simulate-reads simulation.o 
g++ -std=gnu++98 -Wall -I. -I. -Isamtools-1.3/htslib-1.3  -O2 -c -o parseIt.o parseIt.cpp
cd samtools-1.3 && ./configure --without-curses && make -f Makefile samtools
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler defaul

In file included from ./boost/format/alt_sstream.hpp:20:0,
                 from ./boost/format/internals.hpp:23,
                 from ./boost/format.hpp:38,
                 from ./boost/math/policies/error_handling.hpp:31,
                 from ./boost/math/special_functions/gamma.hpp:21,
                 from ./boost/math/special_functions/detail/bessel_jy.hpp:14,
                 from ./boost/math/special_functions/bessel.hpp:18,
                 from ./boost/math/special_functions/airy.hpp:10,
                 from ./boost/math/special_functions.hpp:15,
                 from ./boost/random/generate_canonical.hpp:22,
                 from boost/random.hpp:52,
                 from simul.h:6,
                 from Orientation.h:8,
                 from SingleModel.h:16,
                 from simulation.cpp:22:
./boost/utility/base_from_member.hpp:100:19: warning: variadic templates only available with -std=c++11 or -std=gnu++11
 template<typename ...T>
                   ^~~
./boos

# Gerando Index dos arquivos FASTA

In [7]:
%cd /content/rnaseq-chr22/reference/

/content/rnaseq-chr22/reference


# Instalando bowtie2
> http://bowtie-bio.sourceforge.net/bowtie2/index.shtml

Bowtie 2 is an ultrafast and memory-efficient tool for aligning sequencing reads to long reference sequences.

In [8]:
!wget -c https://sourceforge.net/projects/bowtie-bio/files/bowtie2/2.4.2/bowtie2-2.4.2-sra-linux-x86_64.zip/download

--2021-03-20 04:05:56--  https://sourceforge.net/projects/bowtie-bio/files/bowtie2/2.4.2/bowtie2-2.4.2-sra-linux-x86_64.zip/download
Resolving sourceforge.net (sourceforge.net)... 216.105.38.13
Connecting to sourceforge.net (sourceforge.net)|216.105.38.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.sourceforge.net/project/bowtie-bio/bowtie2/2.4.2/bowtie2-2.4.2-sra-linux-x86_64.zip?ts=1616213156&use_mirror=netactuate&r= [following]
--2021-03-20 04:05:56--  https://downloads.sourceforge.net/project/bowtie-bio/bowtie2/2.4.2/bowtie2-2.4.2-sra-linux-x86_64.zip?ts=1616213156&use_mirror=netactuate&r=
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 216.105.38.13
Connecting to downloads.sourceforge.net (downloads.sourceforge.net)|216.105.38.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://netactuate.dl.sourceforge.net/project/bowtie-bio/bowtie2/2.4.2/bowtie2-2.4.2-sra-linux-x86_64.zi

# Descompactando (unzip)

Descompactando arquivo .ZIP (download)

In [9]:
!unzip download

Archive:  download
   creating: bowtie2-2.4.2-sra-linux-x86_64/
   creating: bowtie2-2.4.2-sra-linux-x86_64/scripts/
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/make_a_thaliana_tair.sh  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/gen_solqual_lookup.pl  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/make_e_coli.sh  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/make_h_sapiens_ncbi36.sh  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/make_c_elegans.sh  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/make_m_musculus_ncbi37.sh  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/infer_fraglen.pl  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/make_rn4.sh  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/make_mm9.sh  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/make_mm10.sh  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/bowtie2-hbb.sh  
  inflating: bowtie2-2.4.2-sra-linux-x86_64/scripts/make_hg19.sh  
  inflating: bowtie2-2.

# Variável de Ambiente
Vamos adicionar o caminho do bowtie2 no PATH.

In [10]:
%cd /content/rnaseq-chr22/reference/bowtie2-2.4.2-sra-linux-x86_64/
%ls

/content/rnaseq-chr22/reference/bowtie2-2.4.2-sra-linux-x86_64
AUTHORS                 bowtie2-build-s*          example/
bowtie2*                bowtie2-build-s-debug*    LICENSE
bowtie2-align-l*        bowtie2-inspect*          MANUAL
bowtie2-align-l-debug*  bowtie2-inspect-l*        MANUAL.markdown
bowtie2-align-s*        bowtie2-inspect-l-debug*  NEWS
bowtie2-align-s-debug*  bowtie2-inspect-s*        README.md
bowtie2-build*          bowtie2-inspect-s-debug*  scripts/
bowtie2-build-l*        BOWTIE2_VERSION           TUTORIAL
bowtie2-build-l-debug*  doc/


In [11]:
# colocando o caminho do diretório bowtie2 na variável PATH
!export PATH=$PWD/bowtie2-2.4.2-sra-linux-x86_64:$PATH

## Executando script makeRef.sh para indexar o arquivo chr22.fa

In [12]:
%%bash
cd /content/rnaseq-chr22/reference/
sh makeRef.sh

rsem-extract-reference-transcripts rsem/rsem 0 chr22.GRCh37.75.gtf None 0 chr22.fa
Parsing gtf File is done!
chr22.fa is processed!
4459 transcripts are extracted.
Extracting sequences is done!
Group File is generated!
Transcript Information File is generated!
Chromosome List File is generated!
Extracted Sequences File is generated!

rsem-preref rsem/rsem.transcripts.fa 1 rsem/rsem
Refs.makeRefs finished!
Refs.saveRefs finished!
rsem/rsem.idx.fa is generated!
rsem/rsem.n2g.idx.fa is generated!

bowtie2-build -f rsem/rsem.idx.fa rsem/rsem
bowtie2-build : No such file or directory!
Please check if you have compiled the associated codes by typing related "make" commands and/or made related executables ready to use.
Mar 20 04:06:03 ..... started STAR run
Mar 20 04:06:03 ... starting to generate Genome files
Mar 20 04:06:04 ..... processing annotations GTF
Mar 20 04:06:05 ... starting to sort Suffix Array. This may take a long time...
Mar 20 04:06:05 ... sorting Suffix Array chunks and savi

In [13]:
# o arquivo rsem.grp deve estar na lista
%ls rsem/

ls: cannot access 'rsem/': No such file or directory


# Voltando para o diretório principal

In [14]:
%cd /content/rnaseq-chr22/

/content/rnaseq-chr22


# Executar STAR

In [15]:
%%bash
cd /content/rnaseq-chr22/
sh run.star.rsem.sh

Mar 20 04:07:04 ..... started STAR run
Mar 20 04:07:04 ..... loading genome
Mar 20 04:07:04 ..... started 1st pass mapping
Mar 20 04:07:36 ..... finished 1st pass mapping
Mar 20 04:07:36 ..... inserting junctions into the genome indices
Mar 20 04:07:38 ..... started mapping
Mar 20 04:08:12 ..... finished mapping
Mar 20 04:08:12 ..... started sorting BAM
Mar 20 04:08:13 ..... finished successfully
rsem-parse-alignments reference/rsem/rsem RNASEQ_data/rsem.A01/rsem.temp/rsem RNASEQ_data/rsem.A01/rsem.stat/rsem RNASEQ_data/A01/Aligned.toTranscriptome.out.bam 3 -tag XM
Done!

rsem-build-read-index 32 1 0 RNASEQ_data/rsem.A01/rsem.temp/rsem_alignable_1.fq RNASEQ_data/rsem.A01/rsem.temp/rsem_alignable_2.fq
Build Index RNASEQ_data/rsem.A01/rsem.temp/rsem_alignable_1.fq is Done!
Build Index RNASEQ_data/rsem.A01/rsem.temp/rsem_alignable_2.fq is Done!

rsem-run-em reference/rsem/rsem 3 RNASEQ_data/rsem.A01/rsem RNASEQ_data/rsem.A01/rsem.temp/rsem RNASEQ_data/rsem.A01/rsem.stat/rsem -p 5
Refs.loa

tcmalloc: large alloc 2773491712 bytes == 0x17126000 @  0x7efce3546887 0x4a31dd 0x4835ce 0x40da73 0x7efce2197bf7 0x418fe5
tcmalloc: large alloc 2773491712 bytes == 0x17f06000 @  0x7f8026d1d887 0x4a31dd 0x4835ce 0x40da73 0x7f802596ebf7 0x418fe5
tcmalloc: large alloc 2773491712 bytes == 0x16b66000 @  0x7f0537a94887 0x4a31dd 0x4835ce 0x40da73 0x7f05366e5bf7 0x418fe5
tcmalloc: large alloc 2773491712 bytes == 0x16efa000 @  0x7f8114e1c887 0x4a31dd 0x4835ce 0x40da73 0x7f8113a6dbf7 0x418fe5
tcmalloc: large alloc 2773491712 bytes == 0x166d2000 @  0x7febbf327887 0x4a31dd 0x4835ce 0x40da73 0x7febbdf78bf7 0x418fe5
tcmalloc: large alloc 2773491712 bytes == 0x16f6c000 @  0x7f58169d9887 0x4a31dd 0x4835ce 0x40da73 0x7f581562abf7 0x418fe5


# Entrar no diretório de Resultados RNASEQ_data

In [16]:
# listando diretórios de output STAR e RSEM
%cd /content/rnaseq-chr22/RNASEQ_data
%ls 

/content/rnaseq-chr22/RNASEQ_data
A01/  A03/  B02/  rsem.A01/  rsem.A03/  rsem.B02/
A02/  B01/  B03/  rsem.A02/  rsem.B01/  rsem.B03/


In [17]:
# listando arquivos do diretorio A01
%cd /content/rnaseq-chr22/RNASEQ_data
%ls -lh A01

/content/rnaseq-chr22/RNASEQ_data
total 16M
-rw-r--r-- 1 root root 5.7M Mar 20 04:08 Aligned.sortedByCoord.out.bam
-rw-r--r-- 1 root root  10M Mar 20 04:08 Aligned.toTranscriptome.out.bam
-rw-r--r-- 1 root root 2.0K Mar 20 04:08 Log.final.out
-rw-r--r-- 1 root root 7.9K Mar 20 04:08 Log.out
-rw-r--r-- 1 root root  447 Mar 20 04:08 Log.progress.out
-rw-r--r-- 1 root root 100K Mar 20 04:08 SJ.out.tab
drwx------ 2 root root 4.0K Mar 20 04:07 _STARgenome/
drwx------ 2 root root 4.0K Mar 20 04:07 _STARpass1/


In [18]:
# listando arquivos do diretorio rsem.A01
%cd /content/rnaseq-chr22/RNASEQ_data
%ls -lh rsem.A01

/content/rnaseq-chr22/RNASEQ_data
total 424K
-rw-r--r-- 1 root root 129K Mar 20 04:08 rsem.genes.results
-rw-r--r-- 1 root root 285K Mar 20 04:08 rsem.isoforms.results
drwxr-xr-x 2 root root 4.0K Mar 20 04:08 rsem.stat/


In [19]:
# copiando resultado de contagem de cada amostra e salvando com o 
# nome da amostra correspondente de cada arquivo.
%%bash
cd /content/rnaseq-chr22/RNASEQ_data 
mkdir -p gene-level
ls -d1 rsem.*  | awk '{ print("cp -v",$1"/rsem.genes.results gene-level/"$1)}'  | sh

'rsem.A01/rsem.genes.results' -> 'gene-level/rsem.A01'
'rsem.A02/rsem.genes.results' -> 'gene-level/rsem.A02'
'rsem.A03/rsem.genes.results' -> 'gene-level/rsem.A03'
'rsem.B01/rsem.genes.results' -> 'gene-level/rsem.B01'
'rsem.B02/rsem.genes.results' -> 'gene-level/rsem.B02'
'rsem.B03/rsem.genes.results' -> 'gene-level/rsem.B03'


In [20]:
# utilizando script R para juntar amostras por gene
%%bash
cd /content/rnaseq-chr22/RNASEQ_data
time R --slave --file=../run.merge.files.R --args gene-level 5 gene-level-5


real	0m0.957s
user	0m0.486s
sys	0m0.199s


In [21]:
# listando tabela merge STAR com todas as amostras por gene 
# Apenas genes do chr22
%%bash
cd /content/rnaseq-chr22/RNASEQ_data
head merge-table-STAR-gene-level-5-50x.txt

symbol	rsem.A01	rsem.A02	rsem.A03	rsem.B01	rsem.B02	rsem.B03
AC002472.13	10	9.22	17.18	11	14	12.1
AC004019.13	23	16	21	16	14	16
AC004471.10	19	26	29	16	11	22
AC006547.14	92	101	81	104	91	76
AC006946.16	49	45	46	42	37	42
AC011718.2	64.69	63.72	52.01	42.66	58.24	43.56
ACR	23	11	19	13	7	16
ADORA2A-AS1	13	27	19	17	9	18
ADRBK2	624	663	640	586	576	556
